# To generate Latex Table
[Go to Result Table](#Result-Table)

In [ ]:
from path import Path
import numpy as np
import matplotlib.pyplot as plt
import argparse
import json
import os
from PIL import Image
import cv2

def compute_errors_test(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25   ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())

    abs_diff = np.mean(np.abs(gt - pred))
    abs_rel = np.mean(np.abs(gt - pred) / gt)

    sq_rel = np.mean(((gt - pred)**2) / gt)

    return abs_rel, abs_diff, sq_rel, rmse, rmse_log, a1, a2, a3

In [ ]:
parser = argparse.ArgumentParser(description='Test octDPSNet ETH3D',
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('data', metavar='DIR',help='path to dataset')
parser.add_argument('--nlabel', type=int ,default=64, help='number of label')
parser.add_argument('--mindepth', type=float ,default=0.5, help='minimum depth')
parser.add_argument('--maxdepth', type=float ,default=10, help='maximum depth')
args = parser.parse_args('DATASET/ETH3D_results/'.split())
args

# Visualize datasets

In [ ]:
root = Path(args.data)
scene_list_path = sorted([name for name in os.listdir(root) if os.path.isdir(os.path.join(root, name))])
scenes = [root/folder for folder in scene_list_path]

In [ ]:
for i, it in enumerate(scene_list_path):
    print(i, it)

In [ ]:
scene = scenes[3]
sequence_set = []
# intrinsics
f_int = open(scene/'gt_cam/cameras.txt', 'r')
lines_int = f_int.readlines()
linelist = lines_int[3].split(' ')
intrinsics = np.array([[float(linelist[4]), 0., float(linelist[6])], [0., float(linelist[5]), float(linelist[7])], [0., 0., 1.]]).astype(np.float32)
intrinsics[0,:] = intrinsics[0,:] * (810/float(linelist[2]))
intrinsics[1,:] = intrinsics[1,:] * (540/float(linelist[3]))
f_int.close()

In [ ]:
intrinsics

In [ ]:
# camera order
f_order = open(scene/'gt_cam/order.txt', 'r')
lines_order = f_order.readlines()
orders = []
for il, line in enumerate(lines_order):
    linelist = line.split(' ')
    orders.append(linelist)

In [ ]:
# camera poses
f_pose = open(scene/'gt_cam/images.txt', 'r')
lines_pose = f_pose.readlines()
linelist_pose = lines_pose[3].split(' ')
ncam = int(linelist_pose[4].split(',')[0])
#poses = [None]*ncam
poses = []
imgidx = [None]*ncam
for il, line in enumerate(lines_pose):
    if il >= 4:
        if il%2 == 0:
            linelist = line.split(' ')
            linelist_ = linelist[1:8]
            imgidx[int(linelist[0])-1] = int((il-4)/2)
            poses.append([float(qt) for qt in linelist_])

In [ ]:
imgs = sorted((scene/'reference_rgb').files('*.png'))
gt_depths = sorted((scene/'gt_depth').files('*.npy'))
gt_demonb = sorted((scene/'DeMoN_best').files('*.npy'))
gt_demonm = sorted((scene/'DeMoN_median').files('*.npy'))
gt_deepmvs = sorted((scene/'DeepMVS').files('*.npy'))
gt_COLMAP = sorted((scene/'COLMAP_unfiltered').files('*.npy'))
gt_COLMAP_filter = sorted((scene/'COLMAP_filtered').files('*.npy'))

depths = gt_depths[0::2]
# demonb = gt_demonb[0::2]
# demonm = gt_demonm[0::2]
# deepmvs = gt_deepmvs[0::2]
# COLMAP = gt_COLMAP[0::2]
# COLMAP_f = gt_COLMAP_filter[0::2]

depths1 = gt_depths[1::2]
# demonb1 = gt_demonb[1::2]
# demonm1 = gt_demonm[1::2]
# deepmvs1 = gt_deepmvs[1::2]
# COLMAP1 = gt_COLMAP[1::2]
# COLMAP1_f = gt_COLMAP_filter[1::2]


In [ ]:
len(imgs)

In [ ]:
idx = -1
img = np.array(Image.open(imgs[idx]))
plt.imshow(img)

In [ ]:
# gt_depths
# gt_demonb
# gt_demonm
# gt_deepmvs
# gt_COLMAP
# gt_COLMAP_filter

In [ ]:
targets = ['octDPSa75','octDPSa9375','DeMoN_best','DeMoN_median','DeepMVS','COLMAP_unfiltered','COLMAP_filtered']

In [ ]:
pred_valid = sorted((scene/'octDPSa75').files('*.npy'))
pred_list = pred_valid[0::2]
valid_list = pred_valid[1::2]

In [ ]:
# inverse depth
gt = np.load(depths[idx])
gt_d = 1/gt
pred = np.load(pred_list[idx])
pred_d = 1/pred
mask = np.load(valid_list[idx])
mask &= ((pred_d>0) & (pred_d<32)) 
mask &= ((gt_d>0) & (gt_d<32)) 

In [ ]:
gt[~mask] = 0
pred[~mask] = 0
fig, ax = plt.subplots(1,2,figsize=(8,8))
ax[0].imshow(gt)
ax[1].imshow(pred)

In [ ]:
compute_errors_test(gt[mask], pred[mask])

In [ ]:
# depth
gt_d[~mask] = 0
pred_d[~mask] = 0
fig, ax = plt.subplots(1,2,figsize=(8,8))
ax[0].imshow(gt_d)
ax[1].imshow(pred_d)

In [ ]:
compute_errors_test(gt_d[mask], pred_d[mask])

In [ ]:
showPointCloudfromNumpy(img, gt_d, intrinsics)
showPointCloudfromNumpy(img, pred_d, intrinsics)

## Batch point cloud visualization

In [ ]:
list(enumerate(targets))

In [ ]:
for i, it in enumerate(scenes):
    print(i, it.basename())

In [ ]:
scene = scenes[4]

In [ ]:
target = 'octDPSa75len4'
target = 'octDPSa75'

pred_valid = sorted((scene/target).files('*.npy'))

imgs = sorted((scene/'reference_rgb').files('*.png'))
gt_depths = sorted((scene/'gt_depth').files('*.npy'))
depths = gt_depths[0::2]
pred_list = pred_valid[0::2]
valid_list = pred_valid[1::2]

for idx in range(0, 15, 3):
    img = np.array(Image.open(imgs[idx]))

    # inverse depth
    gt = np.load(depths[idx])
    gt_d = 1/gt
    pred = np.load(pred_list[idx])
    pred_d = 1/pred
    mask = np.load(valid_list[idx])
    mask &= ((pred_d>0) & (pred_d<32)) 
    mask &= ((gt_d>0) & (gt_d<32)) 

    gt_d[~mask] = 0
    pred_d[~mask] = 0
    
    showPointCloudfromNumpy(img, gt_d, intrinsics)
    showPointCloudfromNumpy(img, pred_d, intrinsics)

# Conversion from test_ETH3D.py

In [ ]:
datafolder = './ETHa75/result_seqlen2_dpsnet_18_checkpoint'
output_name = 'octDPSa75'

datafolder = './a9375len2_dpsnet_19_checkpoint'
output_name = 'octDPSa9375'


datafolder = './a25len2_dpsnet_18_checkpoint/'
output_name = 'octDPSa25'

datafolder = './ETHa75/result_seqlen4_dpsnet_18_checkpoint/'
output_name = 'octDPSa75len4'


datafolder = '/home/komatsu/paper/DPSNet/result_dpsnet_updated/'
output_name = 'DPSNet'

datafolder = './a50len2_dpsnet_19_checkpoint/'
output_name = 'octDPSa50'

datafolder = './a875len2_dpsnet_19_checkpoint/'
output_name = 'octDPSa875'

preds = sorted(Path(datafolder).files('*.npy'))
len(preds)

In [ ]:
cnt = 0
print(output_name)
for scene in scenes:
    print(scene)
    # mkdir
    folder = (scene/output_name)
    folder.mkdir_p()
    
    # gt
    gt_depths = sorted((scene/'gt_depth').files('*.npy'))
    depths = gt_depths[0::2]
    for i, it in enumerate(depths):
        tgt_depth = 1/np.load(it)
        scale = 1/np.amin(tgt_depth[tgt_depth>0])
        
        # pred result
        pred_disp = np.load(preds[cnt])
        pred_depth = args.mindepth*args.nlabel/pred_disp
        pred_depth /= scale
        pred_depth = pred_depth[4:, 22:]
        pred_disp = 1/pred_depth
        
        # save result
        np.save(folder/'{:04d}{}'.format(i,'.npy'), 1/pred_depth)
        np.save(folder/'{:04d}_valid{}'.format(i,'.npy'), np.full((540, 810), True))
        pred_disp /= pred_disp.max()
        pred_disp = (pred_disp*255).clip(0, 255).astype(np.uint8)
        colored_disp = cv2.applyColorMap(pred_disp, cv2.COLORMAP_VIRIDIS)
        cv2.imwrite(folder/'{:04d}{}'.format(i,'.png'), colored_disp)
        cnt += 1

# Compute error

In [ ]:
# check ground truth
max_array = []
min_array = []
for scene in scenes:
    gt_depths = sorted((scene/'gt_depth').files('*.npy'))
    depths = gt_depths[0::2]
    for it in depths:
        gt = np.load(it)
#         print(gt[gt>0].min(), gt[gt>0].max())
#         print(1/gt[gt>0].max(),1/gt[gt>0].min())
        max_array.append(1/gt[gt>0].min())
        min_array.append(1/gt[gt>0].max())
#         max_array.append(gt[gt>0].min())
#         min_array.append(gt[gt>0].max())

In [ ]:
import pandas as pd
pd.DataFrame({'max_array':max_array, 'min_array':min_array}).describe().transpose()

In [ ]:

targets = ['DeMoN_best','DeMoN_median','DeepMVS','COLMAP_unfiltered','COLMAP_filtered']
targets = ['octDPSa75','octDPSa25','octDPSa9375','DPSNet','DeMoN_best','DeMoN_median','DeepMVS','COLMAP_unfiltered','COLMAP_filtered']
targets = ['octDPSa25','octDPSa50','octDPSa75','octDPSa875','octDPSa9375','DPSNet','DeepMVS']

In [ ]:
root = Path(args.data)
scene_list_path = sorted([name for name in os.listdir(root) if os.path.isdir(os.path.join(root, name))])
scenes = [root/folder for folder in scene_list_path]

error_all = {}

for target in targets:
    errors = []
    
    for scene in scenes:
#         print(scene)
        one_scene_errors = []

        gt_depths = sorted((scene/'gt_depth').files('*.npy'))
        depths = gt_depths[0::2]
        depths1 = gt_depths[1::2]

        pred_valid = sorted((scene/target).files('*.npy'))
        pred_list = pred_valid[0::2]
        valid_list = pred_valid[1::2]

        for idx in range(len(depths)):
            # inverse depth
            gt = np.load(depths[idx])
            gt[gt==0]=1e-5
            gt_d = 1/gt
            pred = np.load(pred_list[idx])
            pred[pred==0]=1e-5
            pred_d = 1/pred
            mask = np.load(valid_list[idx])
            mask &= ((pred_d>0) & (pred_d<72)) 
            mask &= ((gt_d>0) & (gt_d<72)) 

            gt[~mask] = 0
            pred[~mask] = 0

            # depth
            gt_d[~mask] = 0
            pred_d[~mask] = 0
            error = compute_errors_test(gt_d[mask], pred_d[mask])
            errors.append(error)
            one_scene_errors.append(error)
        one_scene_errors = np.array(one_scene_errors)
        print("{:14}:{:8.4f}, {:8.4f}, {:8.4f}, {:8.4f}, {:8.4f}, {:8.4f}, {:8.4f}, {:8.4f}".format(
            scene.basename(), *one_scene_errors.mean(0)))


    error_names = ['abs_rel', 'abs_diff', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3']

#     print(target)
    errors = np.array(errors)
    error_all[target] = errors.mean(0)
    print("Depth Results ETH3D: ",target)
    print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format(*error_names))
    print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(*errors.mean(0)))


In [ ]:
error_all

In [ ]:
df = pd.DataFrame(error_all, index=error_names).transpose()
df

In [ ]:
dirname = 'metric_csv_eth'
if not os.path.isdir(dirname):
    os.mkdir(dirname)
print('save directory:', dirname)

for it in df.index:
    fname = '{}/{}.csv'.format(dirname, it)
    print('save csv file in',fname)
    df.loc[it].to_csv(fname)

In [ ]:
tmp = df.loc[it]

In [ ]:
type(tmp)

# Result Table

In [ ]:
import pandas as pd
from collections import OrderedDict
import numpy as np

In [ ]:
csv_fnames = OrderedDict(
    {
    r'Our ($\alpha$=0.25)':'metric_csv_eth/octDPSa25.csv',
    r'Our ($\alpha$=0.50)':'metric_csv_eth/octDPSa50.csv',
    r'Our ($\alpha$=0.75)':'metric_csv_eth/octDPSa75.csv',
    r'Our ($\alpha$=0.875)':'metric_csv_eth/octDPSa875.csv',
    r'Our ($\alpha$=0.9375)':'metric_csv_eth/octDPSa9375.csv',
    'DPSNet':'metric_csv_eth/DPSNet.csv',
    'DeepMVS':'metric_csv_eth/DeepMVS.csv',
    'MVSNet':'metric_csv_eth/MVSNet.csv',
    'R-MVSNet':'metric_csv_eth/RMVSNet.csv',
#     'COLOMAP filter':'metric_csv_eth/COLMAP_filtered.csv',
#     'COLOMAP':'metric_csv_eth/COLMAP_unfiltered.csv',
})
# csv_fnames = OrderedDict(
#     {
#     r'Our ($\alpha$=0.75)':'metric_csv_eth/octDPSa75.csv',
#     'DPSNet':'metric_csv_eth/DPSNet.csv',
#     'DeepMVS':'metric_csv_eth/DeepMVS.csv',
#     'COLOMAP filter':'metric_csv_eth/COLMAP_filtered.csv',
#     'COLOMAP':'metric_csv_eth/COLMAP_unfiltered.csv',
#     'DeMoN':'metric_csv_eth/DeMoN_median.csv',
#     'DeMoN best':'metric_csv_eth/DeMoN_best.csv'
# })


In [ ]:
df_data = OrderedDict({})
for key, fname in csv_fnames.items():
    series = pd.read_csv(fname, header=None, index_col=0, squeeze=True)
    # add
    df_data[key] = series

In [ ]:
df = pd.DataFrame(df_data).transpose()
df

In [ ]:
def _get_add_attr(df, topk, min_is_good, max_is_good):

    add_attr = pd.DataFrame(np.full(df.shape, ""), index=df.index, columns=df.columns)

    _decorate(df, add_attr, min_is_good, topk=topk)
    _decorate(df, add_attr, max_is_good, topk=topk, reverse=True)
    
    return add_attr

def _decorate(df, add_attr, metrics, topk=1, reverse=False):
    for metric in metrics:
        roi = df[metric]
        for i in range(topk)[::-1]:
            sorted_roi = roi.sort_values()
            if reverse:
                sorted_roi = sorted_roi[::-1]
            mask = (roi ==sorted_roi[i])
            add_attr[metric][mask]=i

# Apply style
def highlightDF(df, attrs, min_is_good, max_is_good):
    topk = len(attrs)

    add_attr = _get_add_attr(df, topk, min_is_good, max_is_good)
    
    add_attr.replace(dict(enumerate(attrs)), inplace=True)
    func = lambda x:add_attr
    return df.style.apply(func, axis=None)

In [ ]:
min_is_good = ['abs_rel', 'abs_diff', 'sq_rel', 'rms', 'log_rms']
max_is_good = ['a1', 'a2', 'a3']
# set the style on top K
# 1st: bold, 2nd: underline
attr_bold = 'font-weight: {}'.format('bold')
attr_underline = 'text-decoration:underline'
attrs = [attr_bold, attr_underline]
    
highlightDF(df, attrs, min_is_good, max_is_good)

## To Latex

In [ ]:
topk = 2

colums = ['Abs Rel', 'Abs Diff', 'Sq Rel', 'RMS', 'log RMS',
          '$\delta < 1.25$', '$\delta < 1.25^2$' '& $\delta < 1.25^3$']
tex_attr = {'':'', 0:r'\bfseries', 1:''}
# tex_attr = {'':'', 0:r'\bfseries', 1:r'\underline'}
add_attr = _get_add_attr(df, topk, min_is_good, max_is_good)
add_attr

In [ ]:
df.columns

In [ ]:
print(r'\begin{table*}')
print(r'\caption{xxxxxx}')
print(r'\centering')
print(r'\begin{tabular}{' + '|l|' + 'r'*len(min_is_good) + '|'+'r'*len(max_is_good)+'|' + '}')
print(r'\hline')
print(r'{} &  \multicolumn{5}{c|}{Error (smaller is better)} & \multicolumn{3}{c|}{Threshold (bigger is better)} \\')
print('Model & ' + ' & '.join(colums) + r' \\')
print(r'\hline')
for idx in df.index:
    series = [idx]
    for col in df.columns:
        val = df.loc[idx][col]
        attr = add_attr.loc[idx][col]
        series.append('{:.4f}'.format(val))
        if tex_attr[attr] != '':
            series[-1] = tex_attr[attr] + '{' +series[-1] + '}'
    print(' & '.join(series), r'\\')
print(r'\hline')
print(r'\end{tabular}')
print(r'\label{tab:xxxx}')
print(r'\end{table*}')